# LAB 4 MOS: Método Simplex y sus Variaciones

- Samuel Augusto Hernandez 202213772
- Tomas Alberto Rodriguez 2022128686

## Problema 1: Implementación del Método Simplex Estándar

## Problema 2: Implementación del Método Simplex Dual Phase

## Problema 3: Comparación de Rendimiento con GLPK/Pyomo

## Problema 4: Análisis de Sensibilidad de Programación Lineal